In [1]:
import sqlite3
import csv

### Approach 1 - SQL Query

In [10]:
# Connect to SQLite database
conn = sqlite3.connect('$\\Data Engineer - Assignment Database.db') # your file path
cursor = conn.cursor()

sql_query = """
select c.customer_id, c.age, i.item_name,
sum(case when o.quantity is null then 0 else o.quantity END) AS Quantity
from customers c
join sales s on c.customer_id = s.customer_id
join orders o on s.sales_id = o.sales_id
join items i on o.item_id = i.item_id
where c.age between 18 and 35
group by c.customer_id, i.item_name
having Quantity > 0
order by c.customer_id
"""
# Execute
cursor.execute(sql_query)
# Output
results = cursor.fetchall()
# Output to csv file
with open('output_sql.csv', 'w', newline='') as file:
    writeFile = csv.writer(file, delimiter = ';', lineterminator='\n')
    writeFile.writerow(['Customer_Id', 'Age', 'Item_Name', 'Quantity'])
    for row in results:
        writeFile.writerow(row)
# Close connection
conn.close()
    

### Approach 2 - Using Panda lib 

In [12]:
import pandas as pd

In [22]:
# create connection
conn = sqlite3.connect('$\\Data Engineer - Assignment Database.db') # your file path
# Load data into Dataframes
sales_df = pd.read_sql_query('select * from sales', conn)
customers_df = pd.read_sql_query('select * from customers', conn)
orders_df = pd.read_sql_query('select * from orders', conn)
items_df = pd.read_sql_query('select * from items', conn)

# start merging dataframe
df_merge = pd.merge(customers_df, sales_df, on = 'customer_id')
df_merge = pd.merge(df_merge, orders_df, on = 'sales_id')
df_merge = pd.merge(df_merge, items_df, on = 'item_id')

# Filter customer age
df_filter = df_merge[(df_merge['age'] >= 18) & (df_merge['age'] <= 35)]

# Fill NULL quantities with 0
#df_filter['quantity'] = df_filter['quantity'].fillna(0)

# Convert quantity column to integers
#df_filter['quantity'] = df_filter['quantity'].astype(int)

df_filter['quantity'] = df_filter['quantity'].fillna(0).astype(int)

# Group by and sum of quantity
df_group = df_filter.groupby(['customer_id','age', 'item_name'])['quantity'].sum().reset_index()

# Add condition of Quantity > 0
df_group = df_group[df_group['quantity'] > 0]

# handle float value to integer
#df_group['quantity'] = df_group['quantity'].astype(int)

# write dataframe to csv
df_group.to_csv('output_pandas.csv', sep= ';', index = False, lineterminator = '\n')

# Close database connection
conn.close()